In [25]:
import random
import math
import numpy as np


GOAL_STATE = np.array([[0, 1, 2], 
                       [3, 4, 5], 
                       [6, 7, 8]])


In [26]:

def hamming_distance(state):
    """Calculate the Hamming distance between the current state and goal state."""
    return np.sum(state != GOAL_STATE) - 1  # Subtract 1 to ignore the blank tile

def manhattan_distance(state):
    """Calculate the Manhattan distance between the current state and goal state."""
    distance = 0
    for i in range(3):
        for j in range(3):
            value = state[i][j]
            if value != 0:
                target_x, target_y = divmod(value, 3)
                distance += abs(i - target_x) + abs(j - target_y)
    return distance


In [27]:
def move_blank(state, direction):
    """Move the blank (0) in the given direction (if possible) and return the new state."""
    new_state = state.copy()
    x, y = np.argwhere(state == 0)[0]
    
    if direction == 'up' and x > 0:
        new_state[x, y], new_state[x - 1, y] = new_state[x - 1, y], new_state[x, y]
    elif direction == 'down' and x < 2:
        new_state[x, y], new_state[x + 1, y] = new_state[x + 1, y], new_state[x, y]
    elif direction == 'left' and y > 0:
        new_state[x, y], new_state[x, y - 1] = new_state[x, y - 1], new_state[x, y]
    elif direction == 'right' and y < 2:
        new_state[x, y], new_state[x, y + 1] = new_state[x, y + 1], new_state[x, y]
    
    return new_state

def get_neighbors(state):
    """Return all possible moves for the blank."""
    directions = ['up', 'down', 'left', 'right']
    neighbors = []
    for direction in directions:
        neighbor = move_blank(state, direction)
        if not np.array_equal(neighbor, state):  # Valid move
            neighbors.append(neighbor)
    return neighbors


In [28]:
def hill_climbing(initial_state, heuristic):
    state = initial_state.copy()
    iteration = 0
    max_iterations = 1000
    stuck_at_local_maxima = False
    
    while iteration < max_iterations:
        current_h = heuristic(state)
        neighbors = get_neighbors(state)
        next_state = min(neighbors, key=heuristic)
        next_h = heuristic(next_state)
        
        if next_h >= current_h:  # Local maxima reached
            stuck_at_local_maxima = True
            break
        
        state = next_state
        iteration += 1
        
        if np.array_equal(state, GOAL_STATE):  # Goal reached
            break
    
    return state, current_h, iteration, stuck_at_local_maxima


In [29]:
def simulated_annealing(initial_state, heuristic):
    state = initial_state.copy()
    max_iterations = 1000
    t = 1.0
    T_min = 0.0001
    alpha = 0.99
    iteration = 0
    
    while t > T_min and iteration < max_iterations:
        current_h = heuristic(state)
        neighbors = get_neighbors(state)
        next_state = random.choice(neighbors)
        next_h = heuristic(next_state)
        delta_h = next_h - current_h
        
        if delta_h < 0 or random.uniform(0, 1) < math.exp(-delta_h / t):
            state = next_state
        
        t *= alpha
        iteration += 1
        
        if np.array_equal(state, GOAL_STATE):  # Goal reached
            break
    
    return state, current_h, iteration, t


In [30]:
def read_input_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        # Filter out empty lines and strip any extra whitespace
        lines = [line.strip() for line in lines if line.strip()]
        # Parse the input into a 3x3 array
        initial_state = np.array([list(map(int, line.split())) for line in lines])
        
        # Check if the input is a valid 3x3 matrix
        if initial_state.shape != (3, 3):
            raise ValueError("Input file does not contain a valid 3x3 matrix.")
        
    return initial_state


In [31]:
def log_results(algorithm, initial_state, heuristic_name, heuristic_fn):
    result_state, heuristic_value, iterations, extra_info = algorithm(initial_state, heuristic_fn)
    
    print(f"Algorithm: {algorithm.__name__}")
    print(f"Heuristic: {heuristic_name}")
    print(f"Initial state:\n{initial_state}")
    print(f"Final state:\n{result_state}")
    print(f"Heuristic value: {heuristic_value}")
    print(f"Iterations: {iterations}")
    
    if algorithm == hill_climbing:
        print(f"Stuck at local maxima: {extra_info}")
    elif algorithm == simulated_annealing:
        print(f"Final temperature: {extra_info}")
    print("-" * 40)


In [32]:
def main():
    input_file = "input.txt"
    initial_state = read_input_file(input_file)
    
    print("Running Hill Climbing with Hamming Distance...")
    log_results(hill_climbing, initial_state, "Hamming Distance", hamming_distance)
    
    print("Running Hill Climbing with Manhattan Distance...")
    log_results(hill_climbing, initial_state, "Manhattan Distance", manhattan_distance)
    
    print("Running Simulated Annealing with Hamming Distance...")
    log_results(simulated_annealing, initial_state, "Hamming Distance", hamming_distance)
    
    print("Running Simulated Annealing with Manhattan Distance...")
    log_results(simulated_annealing, initial_state, "Manhattan Distance", manhattan_distance)


main()


Running Hill Climbing with Hamming Distance...
Algorithm: hill_climbing
Heuristic: Hamming Distance
Initial state:
[[3 1 2]
 [6 4 0]
 [7 8 5]]
Final state:
[[0 1 2]
 [3 4 5]
 [6 7 8]]
Heuristic value: 1
Iterations: 5
Stuck at local maxima: False
----------------------------------------
Running Hill Climbing with Manhattan Distance...
Algorithm: hill_climbing
Heuristic: Manhattan Distance
Initial state:
[[3 1 2]
 [6 4 0]
 [7 8 5]]
Final state:
[[0 1 2]
 [3 4 5]
 [6 7 8]]
Heuristic value: 1
Iterations: 5
Stuck at local maxima: False
----------------------------------------
Running Simulated Annealing with Hamming Distance...
Algorithm: simulated_annealing
Heuristic: Hamming Distance
Initial state:
[[3 1 2]
 [6 4 0]
 [7 8 5]]
Final state:
[[0 1 2]
 [3 4 5]
 [6 7 8]]
Heuristic value: 1
Iterations: 22
Final temperature: 0.8016305895390458
----------------------------------------
Running Simulated Annealing with Manhattan Distance...
Algorithm: simulated_annealing
Heuristic: Manhattan Distan